In [57]:
from http import HTTPStatus
import dashscope
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Memory
import json
import openai
import os
import sys
from tenacity import retry, stop_after_attempt, wait_random_exponential
import time
from dotenv import load_dotenv, find_dotenv
import torch
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TextStreamer, GPTQConfig
from kg_rag.config_loader import *
import ast
import requests

/mnt/cephfs/home/chendeyu/anaconda3/envs/KG_RAG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [123]:
import os

# 设置环境变量
os.environ['http_proxy'] = 'http://gpu013:7890'
os.environ['https_proxy'] = 'http://gpu013:7890' # 如果你也需要设置HTTPS代理

# 验证环境变量是否设置成功
print(os.environ['http_proxy'])

http://gpu013:7890


In [ ]:
import os
from pathlib import Path
import sys

# 获取当前工作目录的绝对路径
current_dir = Path.cwd()

# 将配置文件的路径添加到系统路径中
config_path = os.path.join(current_dir, "key_config.py")
if config_path not in sys.path:
    sys.path.append(config_path)

# # 导入 API 密钥
from key_config import OPENAI_KEY
from key_config import DASHSCOPE_KEY

# # 现在可以使用 API_KEY 变量了
print(OPENAI_KEY)
print(DASHSCOPE_KEY)


In [81]:
dashscope.api_key = DASHSCOPE_KEY

system_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":, "problems":, "methods":, "effects":}.Do not use markdown reply'''


def qwen_abstract_extractor(system_prompt, question):
    messages = [{'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': question}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        formatted_response = json.dumps(response, indent=4, ensure_ascii=False)
        print(formatted_response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    return formatted_response
        
abstract = 'General large language models (LLMs), represented by ChatGPT, havedemonstrated significant potential in tasks such as code generation in softwareengineering. This has led to the development of specialized LLMs for softwareengineering, known as Code LLMs. A considerable portion of Code LLMs is derivedfrom general LLMs through model fine-tuning. As a result, Code LLMs are oftenupdated frequently and their performance can be influenced by the base LLMs.However, there is currently a lack of systematic investigation into Code LLMsand their performance. In this study, we conduct a comprehensive survey andanalysis of the types of Code LLMs and their differences in performancecompared to general LLMs. We aim to address three questions: (1) What LLMs arespecifically designed for software engineering tasks, and what is therelationship between these Code LLMs? (2) Do Code LLMs really outperformgeneral LLMs in software engineering tasks? (3) Which LLMs are more proficientin different software engineering tasks? To answer these questions, we firstcollect relevant literature and work from five major databases and open-sourcecommunities, resulting in 134 works for analysis. Next, we categorize the CodeLLMs based on their publishers and examine their relationships with generalLLMs and among themselves. Furthermore, we investigate the performancedifferences between general LLMs and Code LLMs in various software engineeringtasks to demonstrate the impact of base models and Code LLMs. Finally, wecomprehensively maintained the performance of LLMs across multiple mainstreambenchmarks to identify the best-performing LLMs for each software engineeringtask. Our research not only assists developers of Code LLMs in choosing basemodels for the development of more advanced LLMs but also provides insights forpractitioners to better understand key improvement directions for Code LLMs.'

resp = qwen_abstract_extractor(system_prompt, abstract)

{"status_code": 200, "request_id": "fed3b3b0-c335-95f9-82f7-7f0457172c5d", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "```json\n{\n  \"field\": \"large language models\",\n  \"problems\": [\"Lack of systematic investigation into Code LLMs and their performance in software engineering\"],\n  \"methods\": [\n    \"Survey and analysis of Code LLMs, including literature collection, categorization by publisher, comparison with general LLMs, and performance evaluation across tasks\",\n    \"Examination of relationships between Code LLMs and general LLMs\"\n  ],\n  \"effects\": [\n    \"Understanding the types of Code LLMs, their performance in software engineering tasks, and implications for base model selection\",\n    \"Guidance for Code LLM developers on choosing base models and improvements for practitioners\"\n  ]\n}\n```"}}]}, "usage": {"input_tokens": 487, "output_tokens":

In [82]:
# 假设resp是已经正确加载为JSON对象的响应
json_data = json.loads(resp)
print(json_data['output']['choices'][0]['message']['content'])

content_str = json_data['output']['choices'][0]['message']['content'].replace('\n', '').replace('\r', '').replace("```json", "").replace("```", "").strip()

# 现在，解析这个字符串为JSON对象
resp_json = json.loads(content_str)

# 打印解析后的JSON对象中的'field'字段
print(resp_json)
print(resp_json['field'])


```json
{
  "field": "large language models",
  "problems": ["Lack of systematic investigation into Code LLMs and their performance in software engineering"],
  "methods": [
    "Survey and analysis of Code LLMs, including literature collection, categorization by publisher, comparison with general LLMs, and performance evaluation across tasks",
    "Examination of relationships between Code LLMs and general LLMs"
  ],
  "effects": [
    "Understanding the types of Code LLMs, their performance in software engineering tasks, and implications for base model selection",
    "Guidance for Code LLM developers on choosing base models and improvements for practitioners"
  ]
}
```
{'field': 'large language models', 'problems': ['Lack of systematic investigation into Code LLMs and their performance in software engineering'], 'methods': ['Survey and analysis of Code LLMs, including literature collection, categorization by publisher, comparison with general LLMs, and performance evaluation across 

In [115]:
#爬取论文一级消息链接
import urllib.request
import time
import feedparser
from http import HTTPStatus
import dashscope

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = ['all:RL','all:llm', 'all:TTA','all:zero-shot'] # search for electron in all fields
start = 0                       # start at the first result
total_results = 20              # want 20 total results
results_per_iteration = 20       # 5 results at a time
wait_time = 1                   # number of seconds to wait beetween calls

dashscope.api_key = DASHSCOPE_KEY

# system_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":, "problems":, "methods":, "effects":}Please return only the {...} not include json at the beginning and nothing else'''
abstract_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":"...", "problems":"...", "methods":"...", "effects":"..."}.'''


def qwen_abstract_extractor(system_prompt, question):
    messages = [{'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': question}]

    response = dashscope.Generation.call(
        'qwen-72b-chat',
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        formatted_response = json.dumps(response, indent=4, ensure_ascii=False)
        print(formatted_response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    return formatted_response
        
# abstract = 'General large language models (LLMs), represented by ChatGPT, havedemonstrated significant potential in tasks such as code generation in softwareengineering. This has led to the development of specialized LLMs for softwareengineering, known as Code LLMs. A considerable portion of Code LLMs is derivedfrom general LLMs through model fine-tuning. As a result, Code LLMs are oftenupdated frequently and their performance can be influenced by the base LLMs.However, there is currently a lack of systematic investigation into Code LLMsand their performance. In this study, we conduct a comprehensive survey andanalysis of the types of Code LLMs and their differences in performancecompared to general LLMs. We aim to address three questions: (1) What LLMs arespecifically designed for software engineering tasks, and what is therelationship between these Code LLMs? (2) Do Code LLMs really outperformgeneral LLMs in software engineering tasks? (3) Which LLMs are more proficientin different software engineering tasks? To answer these questions, we firstcollect relevant literature and work from five major databases and open-sourcecommunities, resulting in 134 works for analysis. Next, we categorize the CodeLLMs based on their publishers and examine their relationships with generalLLMs and among themselves. Furthermore, we investigate the performancedifferences between general LLMs and Code LLMs in various software engineeringtasks to demonstrate the impact of base models and Code LLMs. Finally, wecomprehensively maintained the performance of LLMs across multiple mainstreambenchmarks to identify the best-performing LLMs for each software engineeringtask. Our research not only assists developers of Code LLMs in choosing basemodels for the development of more advanced LLMs but also provides insights forpractitioners to better understand key improvement directions for Code LLMs.'

# resp = qwen_abstract_extractor(system_prompt, abstract)

print ('Searching arXiv for %s' % search_query)

import json

knowledge_graph = {
    "edges": [],
    "entities": {}
}

# 初始构建知识图谱函数
def build_knowledge_graph(title, author, published, resp, text):
    print('resp:',resp)
    # 如果作者和发布日期还未作为独立的实体加入，则加入它们
    if author not in knowledge_graph["entities"]:
        knowledge_graph["entities"][author] = {}
        knowledge_graph["edges"].append([title, "author", author])
    
    if published not in knowledge_graph["entities"]:
        knowledge_graph["entities"][published] = {}
        knowledge_graph["edges"].append([title, "published", published])
        
    if resp['field'] not in knowledge_graph['entities']:
        knowledge_graph["entities"][resp['field']] = {}
        knowledge_graph["edges"].append([title, "field", resp['field']])
        
    if resp['problems'] not in knowledge_graph['entities']:
        knowledge_graph["entities"][resp['problems']] = {}
        knowledge_graph["edges"].append([title, "problems", resp['problems']])
        
    if resp['methods'] not in knowledge_graph['entities']:
        knowledge_graph["entities"][resp['methods']] = {}
        knowledge_graph["edges"].append([title, "methods", resp['methods']])
        
    if resp['effects'] not in knowledge_graph['entities']:
        knowledge_graph["entities"][resp['effects']] = {}
        knowledge_graph["edges"].append([title, "effects", resp['effects']])
        
        
    # print(resp)
        
    knowledge_graph['entities'][title] = text
    

for j in range(0,len(search_query)):
    for i in range(start,total_results,results_per_iteration):
        
        print ("Results %i - %i" % (i,i+results_per_iteration))
        
        query = 'search_query=%s&start=%i&max_results=%i' % (search_query[j],
                                                            i,
                                                            results_per_iteration)

        # perform a GET request using the base_url and query
        response = urllib.request.urlopen(base_url+query).read()

        # parse the response using feedparser
        feed = feedparser.parse(response)

        # Run through each entry, and print out information
        for entry in feed.entries:
            print ('arxiv-id: %s' % entry.id.split('/abs/')[-1])
            print ('Link:  %s' % entry.id)
            print ('Title:  %s' % entry.title)
            print ('Abstract: %s' %entry.summary.replace("\n",""))
            # feedparser v4.1 only grabs the first author
            authors = ', '.join([person['name'] for person in entry.authors])
            print('Authors:  %s' %authors)
            print ('Publish_time: %s' % entry.published)
            print("=================================")
            # 使用初始函数构建知识图谱

            resp = qwen_abstract_extractor(abstract_prompt, entry.summary.replace("\n",""))
            json_data = json.loads(resp)
            print(json_data['output']['choices'][0]['message']['content'].replace('\n', '').replace('\r', '').replace("```json", "").replace("```", "").strip())
            content_str = json_data['output']['choices'][0]['message']['content'].replace('\n', '').replace('\r', '').replace("```json", "").replace("```", "").strip()
            # 现在，解析这个字符串为JSON对象
            resp_json = json.loads(content_str)
            build_knowledge_graph(entry.title, authors, entry.published, resp_json, entry.summary.replace("\n",""))
        
        # Remember to play nice and sleep a bit before you call
        # the api again!
        print ('Sleeping for %i seconds' % wait_time )
        time.sleep(wait_time)


file_name = 'qwen_72b_chat_KG_llm_TTA_RL_zero-shot.json'
with open(file_name, 'w') as file:
    json.dump(knowledge_graph, file, indent=4)

print(f"Knowledge graph saved to {file_name}.")
# #下载指定链接的pdf
# from urllib import request

# url = "https://arxiv.org/pdf/2311.07989.pdf"
# request.urlretrieve(url,'1.pdf')

Searching arXiv for ['all:RL', 'all:llm', 'all:TTA', 'all:zero-shot']
Results 0 - 20
arxiv-id: 2103.06257v2
Link:  http://arxiv.org/abs/2103.06257v2
Title:  Maximum Entropy RL (Provably) Solves Some Robust RL Problems
Abstract: Many potential applications of reinforcement learning (RL) require guaranteesthat the agent will perform well in the face of disturbances to the dynamics orreward function. In this paper, we prove theoretically that maximum entropy(MaxEnt) RL maximizes a lower bound on a robust RL objective, and thus can beused to learn policies that are robust to some disturbances in the dynamics andthe reward function. While this capability of MaxEnt RL has been observedempirically in prior work, to the best of our knowledge our work provides thefirst rigorous proof and theoretical characterization of the MaxEnt RL robustset. While a number of prior robust RL algorithms have been designed to handlesimilar disturbances to the reward function or dynamics, these methodstypically 

In [104]:
dashscope.api_key = DASHSCOPE_KEY

# system_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":, "problems":, "methods":, "effects":}Please return only the {...} not include json at the beginning and nothing else'''
abstract_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":"...", "problems":"...", "methods":"...", "effects":"..."}.'''


def qwen_get_response(system_prompt, question):
    messages = [{'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': question}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_plus,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        formatted_response = json.dumps(response, indent=4, ensure_ascii=False)
        print(formatted_response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    return formatted_response

In [107]:
#爬取论文一级消息链接
import urllib.request
import time
import feedparser
from http import HTTPStatus
import dashscope

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = ['all:RL','all:llm', 'all:TTA','all:zero-shot'] # search for electron in all fields
start = 0                       # start at the first result
total_results = 20            # want 20 total results
results_per_iteration = 20       # 5 results at a time
wait_time = 1                   # number of seconds to wait beetween calls

dashscope.api_key = DASHSCOPE_KEY

# system_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":, "problems":, "methods":, "effects":}Please return only the {...} not include json at the beginning and nothing else'''
abstract_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":"...", "problems":"...", "methods":"...", "effects":"..."}.'''


def qwen_get_response(system_prompt, question):
    messages = [{'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': question}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_plus,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        formatted_response = json.dumps(response, indent=4, ensure_ascii=False)
        print(formatted_response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    return formatted_response
        
# abstract = 'General large language models (LLMs), represented by ChatGPT, havedemonstrated significant potential in tasks such as code generation in softwareengineering. This has led to the development of specialized LLMs for softwareengineering, known as Code LLMs. A considerable portion of Code LLMs is derivedfrom general LLMs through model fine-tuning. As a result, Code LLMs are oftenupdated frequently and their performance can be influenced by the base LLMs.However, there is currently a lack of systematic investigation into Code LLMsand their performance. In this study, we conduct a comprehensive survey andanalysis of the types of Code LLMs and their differences in performancecompared to general LLMs. We aim to address three questions: (1) What LLMs arespecifically designed for software engineering tasks, and what is therelationship between these Code LLMs? (2) Do Code LLMs really outperformgeneral LLMs in software engineering tasks? (3) Which LLMs are more proficientin different software engineering tasks? To answer these questions, we firstcollect relevant literature and work from five major databases and open-sourcecommunities, resulting in 134 works for analysis. Next, we categorize the CodeLLMs based on their publishers and examine their relationships with generalLLMs and among themselves. Furthermore, we investigate the performancedifferences between general LLMs and Code LLMs in various software engineeringtasks to demonstrate the impact of base models and Code LLMs. Finally, wecomprehensively maintained the performance of LLMs across multiple mainstreambenchmarks to identify the best-performing LLMs for each software engineeringtask. Our research not only assists developers of Code LLMs in choosing basemodels for the development of more advanced LLMs but also provides insights forpractitioners to better understand key improvement directions for Code LLMs.'

# resp = qwen_abstract_extractor(system_prompt, abstract)

print ('Searching arXiv for %s' % search_query)

import json

knowledge_graph = {
    "edges": [],
    "entities": {}
}

# 初始构建知识图谱函数
def build_knowledge_graph(title, author, published, resp):
    print('resp:',resp)
    # 如果作者和发布日期还未作为独立的实体加入，则加入它们
    if author not in knowledge_graph["entities"]:
        knowledge_graph["entities"][author] = {}
        knowledge_graph["edges"].append([title, "author", author])
    
    if published not in knowledge_graph["entities"]:
        knowledge_graph["entities"][published] = {}
        knowledge_graph["edges"].append([title, "published", published])
        
    if resp['field'] not in knowledge_graph['entities']:
        knowledge_graph["entities"][resp['field']] = {}
        
    # print(resp)
        
    knowledge_graph['entities'][title] = resp
    


for i in range(start,total_results,results_per_iteration):
    
    print ("Results %i - %i" % (i,i+results_per_iteration))
    
    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                         i,
                                                        results_per_iteration)

    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()

    # parse the response using feedparser
    feed = feedparser.parse(response)

    # Run through each entry, and print out information
    for entry in feed.entries:
        print ('arxiv-id: %s' % entry.id.split('/abs/')[-1])
        print ('Link:  %s' % entry.id)
        print ('Title:  %s' % entry.title)
        print ('Abstract: %s' %entry.summary.replace("\n",""))
        # feedparser v4.1 only grabs the first author
        print ('First Author:  %s' % entry.author)
        print ('Publish_time: %s' % entry.published)
        print("=================================")
        # print(entry.authors)
        # print('Authors:  %s' %', '.join([person['name'] for person in entry.authors]))
        # 使用初始函数构建知识图谱


    
    # Remember to play nice and sleep a bit before you call
    # the api again!
    print ('Sleeping for %i seconds' % wait_time )
    time.sleep(wait_time)



# #下载指定链接的pdf
# from urllib import request

# url = "https://arxiv.org/pdf/2311.07989.pdf"
# request.urlretrieve(url,'1.pdf')

Searching arXiv for all:eata
Results 0 - 20
arxiv-id: 2403.11491v1
Link:  http://arxiv.org/abs/2403.11491v1
Title:  Uncertainty-Calibrated Test-Time Model Adaptation without Forgetting
Abstract: Test-time adaptation (TTA) seeks to tackle potential distribution shiftsbetween training and test data by adapting a given model w.r.t. any testsample. Although recent TTA has shown promising performance, we still face twokey challenges: 1) prior methods perform backpropagation for each test sample,resulting in unbearable optimization costs to many applications; 2) whileexisting TTA can significantly improve the test performance onout-of-distribution data, they often suffer from severe performance degradationon in-distribution data after TTA (known as forgetting). To this end, we haveproposed an Efficient Anti-Forgetting Test-Time Adaptation (EATA) method whichdevelops an active sample selection criterion to identify reliable andnon-redundant samples for test-time entropy minimization. To allev

In [113]:
search_query = ['all:RL','all:llm', 'all:TTA','all:zero-shot'] # search for electron in all fields
len(search_query)
for j in range(0,len(search_query)):
    print(j)

0
1
2
3
